In [1]:

from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import tensorflow as tf
import random 
from tqdm import tqdm
from PIL import Image, ImageEnhance
from tensorflow.keras.models import model_from_json
import time
class Agent:
    def __init__(self, inputs, outputs):

        # load json and create model
        # json_file = open('modelwack.json', 'r')
        # loaded_model_json = json_file.read()
        # json_file.close()
        # loaded_model = model_from_json(loaded_model_json)

        # self.model = loaded_model
        self.model = Sequential([
            Conv2D(64, (5,5), input_shape=(120,128,1), strides = (2,2), activation='relu'),
            MaxPooling2D(pool_size=(3, 3), strides = (2,2)),
            Conv2D(80, (3,3), activation='relu', strides = (2,2)),
            MaxPooling2D(pool_size=(3, 3), strides = (2,2)),
            Conv2D(128, (2,2), strides = (2,2), activation='relu'),
            MaxPooling2D(pool_size=(3, 3), strides = (2,2)),
            Flatten(),
            Dense(128, activation='relu'),
            Dense(32, activation="relu", name="layer1"),
            Dense(outputs, activation="linear", name="layer2"),
        ])
        self.model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))
        # self.model.load_weights("modelwack3.h5")
        self.memory = []
        print(self.model.summary())
        self.xTrain = []
        self.yTrain = []
        self.lastAction = 0
        self.secondlastAction = 0

    def predict(self, state):
        stateConv = state
        # stateConv = np.squeeze(state).reshape(1,-1)
        qval = self.model.predict(np.reshape(stateConv, (1,120,128,1)))
        return qval
    def act(self, state):
        qval = self.predict(state)
        prob = tf.nn.softmax(tf.math.divide((qval.flatten()), 0.7))
        # print(np.array(prob))
        action = np.random.choice(range(7), p = np.array(prob))
        self.secondlastAction = self.lastAction
        self.lastAction = action
        return action
        

    def remember(self, state, nextState, action, reward, done):
        self.memory.append(np.array([state, nextState, action, reward, done]))

    def learn(self):
        self.batchSize = 32

        if len(self.memory) > 100000:
            self.memory = []
            print("trimming memory")
        if len(self.memory) < self.batchSize:
            print("too little info")
            return #still need to learn, too little memory
        batch = random.sample(self.memory, self.batchSize)
        #check how much time random samples take too
        history = []

        self.learnBatch(batch)

    def learnBatch(self, batch, alpha = 0.9):
        batch = np.array(batch)
        actions = batch[:,2].reshape(self.batchSize).tolist()
        rewards = batch[:,3].reshape(self.batchSize).tolist()

        stateToPredict = batch[:,0].reshape(self.batchSize).tolist()
        nextStateToPredict = batch[:,1].reshape(self.batchSize).tolist()

        statePrediction = self.model.predict(np.reshape(stateToPredict, (self.batchSize, 120, 128,1)))
        nextStatePrediction = self.model.predict(np.reshape(nextStateToPredict, (self.batchSize, 120, 128,1)))
        statePrediction = np.array(statePrediction)
        nextStatePrediction = np.array(nextStatePrediction)

        for i in range(self.batchSize):
            action = actions[i]
            reward = rewards[i]
            nextState = nextStatePrediction[i]
            qval = statePrediction[i, action]
            statePrediction[i, action] += alpha * (reward + 0.95* np.max(nextState) - qval)
            # # doubleq^

        self.xTrain.append(np.reshape(stateToPredict, (self.batchSize, 120, 128,1)))
        self.yTrain.append(statePrediction)
        history = self.model.fit(self.xTrain, self.yTrain, batch_size=5,epochs=1, verbose=0)
        self.xTrain = []
        self.yTrain = []

    def imagePreprocess(self, state):
        # state[0:20] = np.zeros((20,256,3)) + (((self.secondlastAction) / 6)*255)
        state[20:40] = np.zeros((20,256,3)) + (((self.lastAction) / 6)*255)
        img = Image.fromarray(state)
        img = img.resize((120,128),Image.ANTIALIAS)
        bwImg = img.convert('LA')
        # bwImg.show() #use to debug but otherwise this will spam your computer with images
        bwImg = np.array(bwImg, dtype = np.float32)/255
        toRet = np.reshape(bwImg[:,:,0], (120,128,1))
        return toRet



inputs = (240,256,3) #env.observation_space
inputs = 5
outputs = 7 #env.action_space 
agent = Agent(inputs, outputs)
# print(agent.model.summary())
plotX = []
# for i_episode in tqdm(range(200)): #episode counter
for i_episode in tqdm(range(15)): #episode counter
    state = agent.imagePreprocess(env.reset()) #reset the game
    epReward = 0
    for t in (range(500)):
        # start_time2= time.time()
        # action = env.action_space.sample()
        env.render()
        action = agent.act(state)
        totalR = 0
        for _ in range(5): #speeds up learning if you combine various episodes
            nextState, reward, done, info = env.step(action)
            totalR += reward
            if done == True:
                break
        if done == True:
            print("breaking")
            break
        nextState = agent.imagePreprocess(nextState)
        agent.remember(state, nextState, action, totalR, done)
        state = nextState

        epReward += totalR
        if totalR < -9: #consider it episode done
            plotX.append(epReward)
            epReward = 0

        plotX.append(totalR)        

        # 
        # print("--- %s process time seconds ---" % (time.time() - start_time2))
        if t % 12 == 0:
            history = agent.learn()

    # model_json = agent.model.to_json()
    # with open("modelwack3.json", "w") as json_file:
    #     json_file.write(model_json)
    # # serialize weights to HDF5
    # agent.model.save_weights("modelwack3.h5")
    # print("Saved model to disk")
    

0%|          | 0/15 [00:00<?, ?it/s]Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 58, 62, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 28, 30, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 14, 80)        46160     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 80)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         41088     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
flatten (Flatten)   

In [ ]:
for i_episode in tqdm(range(10)): #episode counter
    state = agent.imagePreprocess(env.reset()) #reset the game
    for t in tqdm(range(10000)):
        # start_time2= time.time()
        env.render()
        # action = env.action_space.sample()
        if t % 3 == 0:
            action = agent.act(state)
            nextState, reward, done, info = env.step(action)
            nextState = agent.imagePreprocess(nextState)
            agent.remember(state, nextState, action, reward, done)
        else: 
            nextState, reward, done, _ = env.step(action)
            nextState = agent.imagePreprocess(nextState)

        state = nextState

        if done == True:
            break
        # 
        # print("--- %s process time seconds ---" % (time.time() - start_time2))
        if t % 15 == 0:
            history = agent.learn()